# Sarah Aguasvivas HW 0 Readme

At first, my logic was to use a first order differentiation scheme at $\Omega$ and later worry about the end points. My initial D matrix looked like the following:

\begin{equation*}
D =  \begin{bmatrix}
-1.0/h & 1/h & 0 &... & 0\\
0 & -1/h & 1/h & 0& ...\\
\end{bmatrix}
\end{equation*}

In [ ]:
import numpy
from matplotlib import pyplot

pyplot.style.use('ggplot')

def diffmat(x):
    D=numpy.eye(x.shape[0])
    print(D.shape)
    for i in range(D.shape[0]):
        if i < D.shape[0]-1:
            h= x[i+1]-x[i]
            D[i,i+1] = 1.0/h
            D[i,i] = -1.0/h
        else:
            #if I am in the last point, I am going to 
            #use a second order backwards scheme.
            # 1/(2h) (3*u_i - 4*u_(i-1) + u_(i-2))
            h= x[i]-x[i-2]
            D[i, i]=3.0/h
            D[i, i-1] = -4.0/h
            D[i, i-2] = 1.0/h
            
    return D

n= 20
x= numpy.linspace(-1,1,n)
y=numpy.cos(x) # first derivative
yy=-numpy.sin(x) #second derivative
f= numpy.sin(x) #function in question

DD=diffmat(x)

pyplot.figure()
pyplot.plot(x, numpy.dot(DD, f))
pyplot.plot(x, y)
pyplot.show()
